1) Importo las librerias necesarias

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

2) voy a requerir los años en los que se hizo el mundial, para esto voy a usar un ciclo que me traiga todos los años desde 1930 (inicio del mundial) hasta el 2018 (ultimo mundial que nos interesa) y voy a eliminar los años 1942 y 1946 que son los otros años en los que no se llevo a cabo este evento.

In [2]:
años = []
for i in range(1930,2019,4):
    if i == 1942 or i == 1946:
        continue
    años.append(i)

3) luego voy a requerir la informacion de alguna pagina web, en este caso voy a usar wikipedia, de alli sacare los resultados de cada fase, todo esto lo hare con herramientas de web scraping como beautifulSoup.

hay algo muy interesante y util a la hora de hacer este proyecto, y es que wikipedia posee toda la informacion de todos los mundiales de la misma manera que se va a ver acontinuacion, por lo que puedo hacer una funcion que me traiga año por año la informacion que necesito.

In [3]:
def get_partidos(años):
    respuesta = requests.get(f'https://en.wikipedia.org/wiki/{años}_FIFA_World_Cup')
    contenido = respuesta.text
    soup = BeautifulSoup(contenido, 'lxml')
    #voy a usar "findAll" ya que en el codigo html de la pagina, los datos que nos interesan estan en una clase, por lo que 
    # me interesa traer todos los datos que estan en esa clase y asi me ahorro el trabajo de hacerlo uno por uno.
    matches = soup.findAll('div', class_='footballbox')
    #ahora me interesa de matches sacar los datos de cada 'partido', pero solo me interesa el nombre de los equipos y el resultado, tambien si es visitante o local
    #viendo el html y estos datos estan en el 'th' y en la clase 'fhome'(local) 'fscore'(resultado) y 'faway'(visitante)
    local = []
    resultado = []
    visitante = []
    for match in matches:
        local.append(match.find('th', class_='fhome').get_text())
        resultado.append(match.find('th', class_='fscore').get_text())
        visitante.append(match.find('th', class_='faway').get_text())
    partidos = {'local': local, 'resultado': resultado, 'visitante': visitante}
    df_partidos = pd.DataFrame(partidos)
    df_partidos['año'] = años
    return df_partidos

una vez hecha la funcion, voy a ejecutarla con los años hechos en el paso 2, y voy a guardar la informacion en un dataframe, para luego exportarlo a un archivo csv.

In [4]:
mundiales = [get_partidos(año) for año in años]
df_mundiales = pd.concat(mundiales, ignore_index=True)

In [5]:
df_mundiales.to_csv('mundiales.csv', index=False)

In [6]:
#tambien sacaremos los partidos del mundial pasado
respuesta = requests.get(f'https://web.archive.org/web/20221113165455/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
contenido = respuesta.text
soup = BeautifulSoup(contenido, 'lxml')
matches = soup.findAll('div', class_='footballbox')
local = []
resultado = []
visitante = []
for match in matches:
        local.append(match.find('th', class_='fhome').get_text())
        resultado.append(match.find('th', class_='fscore').get_text())
        visitante.append(match.find('th', class_='faway').get_text())
partidos = {'local': local, 'resultado': resultado, 'visitante': visitante}
df_mundialQatar = pd.DataFrame(partidos)
df_mundialQatar['año'] = 2022
    

df_mundialQatar.to_csv('mundialQatar.csv', index=False)

In [26]:
df_mundiales['año'].value_counts().sort_index()

1930    18
1934    17
1938    19
1950    22
1954    26
1958    35
1962    32
1966    32
1970    32
1974    38
1978    38
1982    52
1986    52
1990    52
1994    52
1998    64
2002    64
2006    64
2010    64
2014    64
2018    64
Name: año, dtype: int64